In [ ]:
import numpy as np
from scipy.special import jv
from scipy.linalg import inv
from DeepFMKit.plotting import default_rc
import matplotlib.pyplot as plt
plt.rcParams.update(default_rc)

def calculate_jacobian(ndata, param):
    """
    Calculates the Jacobian matrix (J) of the DFMI model.
    (This function remains unchanged from the previous version)
    """
    a, m, phi, psi = param
    J = np.zeros((2 * ndata, 4))
    j = np.arange(1, ndata + 1)
    
    phase_term = np.cos(phi + j * np.pi / 2.0)
    cos_jpsi = np.cos(j * psi)
    sin_jpsi = np.sin(j * psi)
    
    bessel_j = jv(j, m)
    bessel_deriv = 0.5 * (jv(j - 1, m) - jv(j + 1, m))
    
    common_term = a * phase_term * bessel_j
    model_q = common_term * cos_jpsi
    model_i = -common_term * sin_jpsi
    
    if a != 0:
        J[:ndata, 0] = model_q / a
        J[ndata:, 0] = model_i / a

    common_deriv_term_m = a * phase_term * bessel_deriv
    J[:ndata, 1] = common_deriv_term_m * cos_jpsi
    J[ndata:, 1] = -common_deriv_term_m * sin_jpsi

    phase_deriv_term = np.cos(phi + j * np.pi / 2.0 + np.pi / 2.0)
    common_deriv_term_phi = a * phase_deriv_term * bessel_j
    J[:ndata, 2] = common_deriv_term_phi * cos_jpsi
    J[ndata:, 2] = -common_deriv_term_phi * sin_jpsi

    J[:ndata, 3] = common_term * -sin_jpsi * j
    J[ndata:, 3] = -common_term * cos_jpsi * j
    
    return J

def calculate_m_precision(m_range, ndata, snr_db):
    """
    Core calculation function for statistical uncertainty of 'm'.
    
    Parameters
    ----------
    m_range : array_like
        The range of modulation depths 'm' to analyze.
    ndata : int
        Number of harmonics to use in the fit.
    snr_db : float
        Signal-to-Noise Ratio in dB for the I/Q measurements.

    Returns
    -------
    numpy.ndarray
        An array of the statistical uncertainty (delta_m) for each m in m_range.
    """
    param_fixed = np.array([1.0, 0, np.pi/4, 0.0]) # a, m, phi, psi
    snr_linear = 10**(snr_db / 20.0)
    noise_variance = (1.0 / snr_linear)**2
    
    delta_m_list = []
    for m_true in m_range:
        param_fixed[1] = m_true
        J = calculate_jacobian(ndata, param_fixed)
        JTJ = J.T @ J
        try:
            covariance_matrix = noise_variance * inv(JTJ)
            delta_m = np.sqrt(covariance_matrix[1, 1])
            delta_m_list.append(delta_m)
        except np.linalg.LinAlgError:
            delta_m_list.append(np.inf)
            
    return np.array(delta_m_list)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.cm import viridis

def plot_precision_vs_ndata():
    """
    Plots the statistical uncertainty in 'm' as a function of the number of
    harmonics used in the fit.
    """
    # --- Analysis Parameters ---
    snr_db_fixed = 80.0
    m_range = np.linspace(0.1, 30.0, 500)
    ndata_range = np.arange(3, 31)
    
    # --- Plotting Setup ---
    fig, ax = plt.subplots(figsize=(6, 3), dpi=300)
    colors = viridis(np.linspace(0, 1, len(ndata_range)))

    print("Calculating uncertainty for different numbers of harmonics...")
    for i, ndata in enumerate(ndata_range):
        delta_m = calculate_m_precision(m_range, ndata, snr_db_fixed)
        ax.semilogy(m_range, delta_m, color=colors[i], label=f'ndata = {ndata}')

    # --- Aesthetics ---
    ax.set_xlabel('Modulation Depth (m)')
    ax.set_ylabel(r'Statistical Uncertainty ($\delta m$)')
    ax.set_title(f'DFMI Precision vs. Number of Harmonics (SNR = {snr_db_fixed} dB)')
    ax.grid(True, which='both', linestyle=':')
    # ax.set_ylim(1e-7, 1e-2) # Set a reasonable y-axis limit
    
    # Create a colorbar for the legend
    sm = plt.cm.ScalarMappable(cmap=viridis, norm=plt.Normalize(vmin=ndata_range.min(), vmax=ndata_range.max()))
    sm.set_array([])
    cbar = fig.colorbar(sm, ax=ax, ticks=ndata_range[::3]) # Show ticks every 3rd value
    cbar.set_label('Number of Harmonics (ndata)', rotation=270, labelpad=20)

    plt.tight_layout()
    return ax

ax = plot_precision_vs_ndata()
ax.set_ylim(0,1e-2)
plt.show()

In [ ]:
def plot_precision_vs_snr(ndata):
    """
    Plots the statistical uncertainty in 'm' as a function of the signal-to-noise
    ratio (SNR) of the measurement.
    """
    # --- Analysis Parameters ---
    ndata_fixed = ndata
    m_range = np.linspace(1.0, 30.0, 500)
    snr_db_range = np.linspace(20, 100, 11) # From 40 dB to 100 dB
    
    # --- Plotting Setup ---
    fig, ax = plt.subplots(figsize=(12, 7))
    colors = viridis(np.linspace(0, 1, len(snr_db_range)))

    print("Calculating uncertainty for different SNR values...")
    for i, snr_db in enumerate(snr_db_range):
        delta_m = calculate_m_precision(m_range, ndata_fixed, snr_db)
        ax.semilogy(m_range, delta_m, color=colors[i], label=f'SNR = {snr_db} dB')
        ax.axhline(y=np.sqrt(8)/10**(snr_db/20))

    # --- Aesthetics ---
    ax.set_xlabel('Modulation Depth (m)', fontsize=14)
    ax.set_ylabel(r'Statistical Uncertainty ($\delta m$)', fontsize=14)
    ax.set_title(f'DFMI Precision vs. Signal-to-Noise Ratio (ndata = {ndata_fixed})', fontsize=16)
    ax.grid(True, which='both', linestyle=':')
    # ax.set_ylim(1e-8, 1e-2)
    ax.legend(loc='upper right')
    
    plt.tight_layout()
    plt.show()

if __name__ == '__main__':
    plot_precision_vs_snr(100)